In [1]:
from google.colab import drive, files
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [3]:
def getLocalFiles():
    _files = files.upload()
    if _files:
        for k, v in _files.items():
            open(k, 'wb').write(v)

getLocalFiles()

Saving my_applications.py to my_applications.py
Saving my_data.py to my_data.py
Saving my_gradcam.py to my_gradcam.py
Saving my_model.py to my_model.py


In [2]:
import tensorflow as tf
import numpy as np
import os
import imgaug as aug

from keras.models import load_model
from my_data import *
from my_model import *
from my_applications import *
from my_gradcam import *

Using TensorFlow backend.


In [0]:
ROOT_PATH = 'drive/My Drive/master1/medical_image_recognition/'
DATASET_NAME = 'chest_xray'
DATASET_PATH = ROOT_PATH + 'datasets/' + DATASET_NAME + '/'
ARRAY_PATH = ROOT_PATH + 'arrays/' + DATASET_NAME + '/'
MODEL_PATH = ROOT_PATH + 'models/' + DATASET_NAME + '/'

HEIGHT, WIDTH, CHANNELS = 224, 224, 3

# the next instructions are used to make results reproducible
SEED = 1234
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(SEED)
tf.set_random_seed(SEED)
aug.seed(SEED)

In [4]:
dataclass = data_helper(DATASET_NAME,
                        DATASET_PATH,
                        ARRAY_PATH,
                        height=HEIGHT,
                        width=WIDTH,
                        channels=CHANNELS,
                        histogram_equalization=False)

print(dataclass.directories)
print(dataclass.labels)

['test', 'train', 'val']
{'NORMAL': 0, 'PNEUMONIA': 1}


In [5]:
#dataclass.save_images()
dataclass.get_images()

train_x = dataclass.x['train']
train_y = dataclass.y['train']
test_x  = dataclass.x['test']
test_y  = dataclass.y['test']
val_x   = dataclass.x['val']
val_y   = dataclass.y['val']

test: getting images from drive/My Drive/master1/medical_image_recognition/arrays/chest_xray/test(224,224,3).h5
Data shape : (624, 224, 224, 3)
Labels shape : (624, 2)
Found 624 images in 8 seconds.
train: getting images from drive/My Drive/master1/medical_image_recognition/arrays/chest_xray/train(224,224,3).h5
Data shape : (5221, 224, 224, 3)
Labels shape : (5221, 2)
Found 5221 images in 46 seconds.
val: getting images from drive/My Drive/master1/medical_image_recognition/arrays/chest_xray/val(224,224,3).h5
Data shape : (16, 224, 224, 3)
Labels shape : (16, 2)
Found 16 images in 0 seconds.


In [6]:
import pandas as pd
df = pd.DataFrame(train_y, columns=dataclass.labels.keys())

for column in df.columns:
    print(f'{column} : {np.array(df[df[column] == 1]).sum()}')

NORMAL : 1346.0
PNEUMONIA : 3875.0


In [0]:
class_weight_mapping = {0: 1.0, 1: 0.4}

In [10]:
# optimizers
from keras.optimizers import Adam
optimizer = Adam(lr=1e-6, decay=1e-5)

weights_path = ROOT_PATH + 'vgg19_weights_no_top.h5'

#vgg19_ = vgg19.VGG19(weights='imagenet', include_top=False)
#vgg19_.save_weights(weights_path)

# create the model
model = create_our_cnn(dataclass.num_labels,
                       weights_path=weights_path,
                       input_shape=(dataclass.height,
                                    dataclass.width,
                                    dataclass.channels),
                       optimizers=optimizer)

# paths
save_model_path = MODEL_PATH + model.name + '.hdf5'
save_history_path = MODEL_PATH + model.name + '.history'

Model created.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_sepconv1 (SeparableCo (None, 112, 112, 128)     8896      
_________________________________________________________________
block2_sepconv2 (SeparableCo (None, 112, 112, 128)     17664     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [0]:
#for layer in model.layers:
#    print(f'{layer.name} : {layer.trainable}')

In [11]:
# training
train_model(model,
            train_x,
            train_y,
            test_x,
            test_y,
            save_model_path,
            epochs=300,
            batch_size=32,
            metric='val_loss',
            save_best_only=True,
            save_weights_only=False,
            stop_after=20,
            save_history=True,
            save_history_path=save_history_path,
            class_weight_mapping=class_weight_mapping)

Train on 5221 samples, validate on 624 samples
Epoch 1/300
5221/5221 [==============================] - 79s 15ms/step - loss: 0.3841 - acc: 0.7878 - val_loss: 0.6909 - val_acc: 0.8285

Epoch 00001: val_loss improved from inf to 0.69094, saving model to drive/My Drive/master1/medical_image_recognition/models/chest_xray/our_cnn.hdf5
Epoch 2/300
5221/5221 [==============================] - 79s 15ms/step - loss: 0.3808 - acc: 0.8497 - val_loss: 0.6824 - val_acc: 0.7596

Epoch 00002: val_loss improved from 0.69094 to 0.68237, saving model to drive/My Drive/master1/medical_image_recognition/models/chest_xray/our_cnn.hdf5
Epoch 3/300
5221/5221 [==============================] - 80s 15ms/step - loss: 0.3673 - acc: 0.8506 - val_loss: 0.6572 - val_acc: 0.7292

Epoch 00003: val_loss improved from 0.68237 to 0.65719, saving model to drive/My Drive/master1/medical_image_recognition/models/chest_xray/our_cnn.hdf5
Epoch 4/300
5221/5221 [==============================] - 81s 15ms/step - loss: 0.3331 -

(<keras.engine.training.Model at 0x7f04e34a51d0>,
 <keras.callbacks.History at 0x7f04e3aabac8>)